In [4]:
import pandas as pd
import numpy as np
import nltk
from pickle import load
from sklearn.pipeline import Pipeline, make_pipeline

# Loading models

In [2]:
tfidf = load(open('tfidf', 'rb'))
model = load(open('model', 'rb'))

# Loading stopwords

In [5]:
nltk.download('stopwords')
tokenizer = nltk.RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mlmaster/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loading Data

In [6]:
x = pd.read_csv('QueryResults.csv')[0:5]
title = x['Title']
body = x['Body']

# Building the pipeline

In [ ]:
def pipe(title, body):
    body = title + body
    body = title + body
    parser = lambda x: html.fromstring(x).text_content()
    body = list(map(parser, body))
    body.replace(r'\n', '', regex=True, inplace=True)

    # There is still some html tags, let's remove them with a regex
    TAG_RE = re.compile(r'<[^>]+>')
    tags_re = lambda x: TAG_RE.sub('', x)
    body = list(map(tags_re, body))
    
    
    

In [12]:
pipe = Pipeline([('tf-idf', tfidf), ('Model', model)])